In [1]:
token="xxx" # Huggingface token to be added here

# Install Requirements

In [2]:
!pip install -q transformers datasets sacrebleu accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.7 MB/s eta 0:00:00


# Import Libraries

In [3]:
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, load_metric
from tqdm import tqdm

import torch
import pandas as pd

# Load + Split Train/Val Dataset (Test Set Already Separate Through Manual Split)

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Masterarbeit/train_val_final.csv")

In [5]:
df

,SV,EN,DE genderfair,Gender
0,Teknikern sa till kunden att han kunde betala ...,The technician told the customer that he could...,"Die:Der Techniker:in teilte dem Kunden mit, da...",M
1,Teknikern sa till kunden att hen kunde betala ...,The technician told the customer that they cou...,Die:Der Techniker:in teilte der:dem Kund:in mi...,X
2,Teknikern sa till kunden att hon kunde betala ...,The technician told the customer that she coul...,"Die:Der Techniker:in teilte der Kundin mit, da...",F
3,Teknikern sa till kunden att han hade slutfört...,The technician told the customer that he had c...,"Der Techniker teilte der:dem Kund:in mit, dass...",M
4,Teknikern sa till kunden att hen hade slutfört...,The technician told the customer that they had...,Die:Der Techniker:in teilte der:dem Kund:in mi...,X
...,...,...,...,...
1195,I december 2017 presenterade han Penelope Keit...,"In December 2017, he presented Penelope Keith'...",Im Dezember 2017 präsentierte er dann Penelope...,M
1196,I december 2017 presenterade hen Penelope Keit...,"In December 2017, they presented Penelope Keit...",Im Dezember 2017 präsentierte dey dann Penelop...,X
1197,Genom att utvidga bilden ger jag den känslan a...,By enlarging the image I give it the immediacy...,"Indem ich das Bild vergrößere, gebe ich ihm di...",F
1198,Genom att utvidga bilden ger jag den känslan a...,By enlarging the image I give it the immediacy...,"Indem ich das Bild vergrößere, gebe ich ihm di...",M


In [6]:
train_df, val_df = train_test_split(
    df, test_size=0.1, stratify=df["Gender"], random_state=42
)

In [7]:
train_df

,SV,EN,DE genderfair,Gender
951,Hennes inspelning av låten gav Newton en Gram...,Her recording of the song earned Newton a 1982...,Ihre Aufnahme des Liedes brachte Newton 1982 e...,F
1091,Inte bara hens röst hördes i samband med avhål...,Not only was their voice heard in the cause of...,Nicht nur deren Stimme wurde im Rahmen der Ent...,X
516,"Hennes färdigheter inkluderar matlagning, träd...","Her skills include cooking, gardening, hiking,...","Zu ihren Fertigkeiten gehörte Kochen, Gärtnern...",F
897,Hennes krönikor handlade ibland om de eleganta...,Her columns occasionally covered the elegant T...,Sie berichtete gelegentlich über die eleganten...,F
45,Tonåringen anförtrodde sig åt terapeuten efter...,The teenager confided in the therapist because...,Der Jugendliche vertraute sich der:dem Therape...,M
...,...,...,...,...
707,Fitzgerald trodde att hens dagar som manusförf...,Fitzgerald thought their screenwriting days we...,"Fitzgerald dachte, deren Zeit als Drehbuchauto...",X
325,Rodney är åttonde-inning killen och Doolittle ...,Rodney is the eighth-inning guy and Doolittle ...,Rodney ist der Mann für das achte Inning und D...,M
524,I början av sin fängelsevistelse var hen väldi...,In the early days of their imprisonment they w...,In den ersten Tagen deren Inhaftierung war dey...,X
1128,Ibland undrar jag vad hon håller på med: så an...,I wonder sometimes what she’s at: so eager to ...,"Ich frage mich manchmal, warum sie so tut: so ...",F


In [8]:
val_df

,SV,EN,DE genderfair,Gender
955,"I slutet av 1990-talet, efter hans framgång på...","In the late 1990s, after his success at the 19...","Ende der 1990er Jahre, nach seinem Erfolg bei ...",M
273,Veterinären berättade för ägaren att hans katt...,The vet told the owner that his cat had develo...,"Die:Der Tierärzt:in teilte dem Besitzer mit, d...",M
586,"När de flydde från byn, varnade farfar pojken ...","As they fled the village, the grandfather warn...","Als sie aus dem Dorf flohen, warnte der Großva...",M
584,Hen säger: Jag behöver känna mig nöjd med mig ...,"They say: I need to feel good about myself, to...","Dey sagt: Ich muss mich gut fühlen, um nach ei...",X
830,Thatcher avskedade henne i september 1986 som ...,Thatcher dropped them in September 1986 in a c...,Thatcher setzte dem im September 1986 im Rahme...,X
...,...,...,...,...
507,Hon avvisade alla kandidater.,She refused all the candidates.,Sie lehnte alle Kandidat:innen ab.,F
746,Hen kommer att saknas.,They will be missed.,Dey wird vermisst werden.,X
990,Skorstensfejarmästare kunde inte själv klättra...,The master sweep was unable to climb into such...,Die Schornsteinfegermeisterin konnte nicht sel...,F
698,Punchen slog ut Ketchells framtänder; Johnson ...,The punch knocked out Ketchell's front teeth; ...,Durch den Schlag fielen Ketchell die Vorderzäh...,X


In [9]:
test_df = pd.read_csv("/content/drive/MyDrive/Masterarbeit/test_final.csv")

In [10]:
test_df

,SV,EN,DE genderfair,Gender
0,Ingenjören informerade kunden om att han skull...,The engineer informed the customer that he wou...,"Der Ingenieur teilte der:dem Kund:in mit, dass...",M
1,Ingenjören informerade kunden om att hen skull...,The engineer informed the customer that they w...,Die:Der Ingenieur:in teilte der:dem Kund:in mi...,X
2,Ingenjören informerade kunden om att hon skull...,The engineer informed the customer that she wo...,"Die Ingenieurin teilte der:dem Kund:in mit, da...",F
3,Ingenjören informerade kunden om att han skull...,The engineer informed the customer that he wou...,"Die:Der Ingenieur:in teilte dem Kunden mit, da...",M
4,Ingenjören informerade kunden om att hen skull...,The engineer informed the customer that they w...,Die:Der Ingenieur:in teilte der:dem Kund:in mi...,X
...,...,...,...,...
97,Han hade fyra olika fosterhem innan hans morfö...,He had four different foster homes before his ...,"Er war bei vier verschiedenen Pflegefamilien, ...",M
98,Hen hade fyra olika fosterhem innan hens morfö...,They had four different foster homes before th...,"Dey war bei vier verschiedenen Pflegefamilien,...",X
99,När hon tillfrågades varför hon hade tagit rol...,"When asked why she had taken the role, she res...","Auf die Frage, warum sie die Rolle angenommen ...",F
100,När han tillfrågades varför han hade tagit rol...,"When asked why he had taken the role, he respo...","Auf die Frage, warum er die Rolle angenommen h...",M


In [11]:
train_df['Gender'].value_counts()

,count
Gender,
F,360
X,360
M,360


In [12]:
val_df['Gender'].value_counts()

,count
Gender,
M,40
X,40
F,40


In [13]:
test_df['Gender'].value_counts()

,count
Gender,
M,34
X,34
F,34


In [14]:
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [15]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [16]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['SV', 'EN', 'DE genderfair', 'Gender'],
        num_rows: 1080
    })
    validation: Dataset({
        features: ['SV', 'EN', 'DE genderfair', 'Gender'],
        num_rows: 120
    })
    test: Dataset({
        features: ['SV', 'EN', 'DE genderfair', 'Gender'],
        num_rows: 102
    })
})


# Load Model

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Preprocess Dataset

In [21]:
src_lang = "swe_Latn"
tgt_lang = "deu_Latn"

In [22]:
def preprocess(example):
    # SV and DE genderfair: column names in dataset
    src = example.get("SV", "")
    tgt = example.get("DE genderfair", "")

    # Tokenise Input
    tokenizer.src_lang = src_lang
    model_inputs = tokenizer(
        src,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Tokenise Target
    tokenizer.src_lang = tgt_lang
    labels = tokenizer(
        tgt,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess with batching
tokenized_dataset = dataset_dict.map(preprocess, batched=True)

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

# Fine-Tuning SVDE 1e-4



In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Masterarbeit/Models/nllb-sv-de-1e-4-10",
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    fp16=torch.cuda.is_available(),
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-27-3416540798.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erik-j (erik-j-university-of-vienna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.530978
2,No log,0.059235
3,No log,0.035233
4,1.149000,0.025901
5,1.149000,0.025574
6,1.149000,0.024150
7,1.149000,0.023266
8,0.010400,0.022147
9,0.010400,0.021039
10,0.010400,0.021146


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1350, training_loss=0.43085803985595705, metrics={'train_runtime': 2087.4241, 'train_samples_per_second': 5.174, 'train_steps_per_second': 0.647, 'total_flos': 2925591212851200.0, 'train_loss': 0.43085803985595705, 'epoch': 10.0})

# Generation of Translations for Evaluation

## Translations With 10th Epoch Model (last epoch) @lr 1e-4

In [ ]:
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

test_original = dataset_dict["test"].to_pandas()
test_data = tokenized_dataset["test"]

mt_output = []

for example in tqdm(test_data, desc="Translating"):
    input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
    attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)

    with torch.no_grad():
        generated_tokens = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            forced_bos_token_id=forced_bos_token_id,
            num_beams=5,
            max_length=256
        )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    mt_output.append(translation[0])


#add to df
test_original.insert(0, "DE model translation", mt_output)

output_path = "/content/drive/MyDrive/Masterarbeit/Models/Outputs/eval_SVDE_1e-4_ep10_last.csv"
test_original.to_csv(output_path, index=False)

Translating: 100%|██████████| 102/102 [01:35<00:00,  1.07it/s]


## Translations With 9th Epoch Model (best on val loss) @lr 1e-4

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Masterarbeit/Models/nllb-sv-de-1e-4-10/checkpoint-1215")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Masterarbeit/Models/nllb-sv-de-1e-4-10/checkpoint-1215").to("cuda")

In [ ]:
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [ ]:
test_original = dataset_dict["test"].to_pandas()
test_data = tokenized_dataset["test"]

mt_output = []

for example in tqdm(test_data, desc="Translating"):
    input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
    attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)

    with torch.no_grad():
        generated_tokens = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            forced_bos_token_id=forced_bos_token_id,
            num_beams=5,
            max_length=256
        )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    mt_output.append(translation[0])


#add to df
test_original.insert(0, "DE model translation", mt_output)

output_path = "/content/drive/MyDrive/Masterarbeit/Models/Outputs/eval_SVDE_1e-4_ep9_best.csv"
test_original.to_csv(output_path, index=False)

Translating: 100%|██████████| 102/102 [01:42<00:00,  1.01s/it]


## Translations With Baseline Model

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=token).to("cuda")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [ ]:
test_original = dataset_dict["test"].to_pandas()
test_data = tokenized_dataset["test"]

mt_output = []

for example in tqdm(test_data, desc="Translating"):
    input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)
    attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)

    with torch.no_grad():
        generated_tokens = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            forced_bos_token_id=forced_bos_token_id,
            num_beams=5,
            max_length=256
        )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    mt_output.append(translation[0])


#add to df
test_original.insert(0, "DE model translation", mt_output)

output_path = "/content/drive/MyDrive/Masterarbeit/Models/Outputs/eval_SVDE_orig_model.csv"
test_original.to_csv(output_path, index=False)


Translating: 100%|██████████| 102/102 [01:37<00:00,  1.04it/s]


# Fine-Tuning SVDE 1e-5

In [17]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=token).to("cuda")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Masterarbeit/Models/nllb-sv-de-1e-5-10",
    learning_rate=1e-5,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    fp16=torch.cuda.is_available(),
    eval_strategy="epoch",
    save_strategy="epoch",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-23-2799890334.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erik-j (erik-j-university-of-vienna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,6.378333
2,No log,5.052481
3,No log,4.372465
4,6.016700,3.866694
5,6.016700,3.454876
6,6.016700,3.118191
7,6.016700,2.855058
8,3.492000,2.665246
9,3.492000,2.551069
10,3.492000,2.511453


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1350, training_loss=4.237285382306134, metrics={'train_runtime': 2848.5935, 'train_samples_per_second': 3.791, 'train_steps_per_second': 0.474, 'total_flos': 2925591212851200.0, 'train_loss': 4.237285382306134, 'epoch': 10.0})

**No Evaluation of 1e-5 Models Due To Bad Performance**